In [2]:
import math
import re
import time
from tqdm import trange, tqdm
import json
import re
import requests
import sys, os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from pprint import pprint

from selenium import webdriver

normalize_pattern = re.compile('[\r\n\t]')
doublespcae_pattern = re.compile('[\s]+')

## 영화 idx 크롤링

In [54]:
current_dir = os.getcwd()
df = pd.read_csv(current_dir+"/raw_data/original_title.csv")

In [74]:
current_dir = os.getcwd()

df2 = pd.read_csv(current_dir+"/new_data/movie_in_Naver.csv")

In [75]:
df.head()

,Unnamed: 0,id,title,type,original_title,idx
0,0,0,모가디슈,Movie,NaN,192150.0
1,1,1,베놈,Movie,Venom,187323.0
2,2,2,스파이더맨,Movie,Spider-Man,208077.0
3,3,3,스파이더맨: 뉴 유니버스,Movie,Spider-Man: Into the Spider-Verse,171725.0
4,4,4,스파이더맨: 홈커밍,Movie,Spider-Man: Homecoming,135874.0


In [199]:
id_title = df2[['id', 'title','idx']]

In [76]:
df2.head()

,id,title,type,original_title,idx
0,0,모가디슈,Movie,NaN,192150
1,1,베놈,Movie,Venom,119428
2,2,스파이더맨,Movie,Spider-Man,31307
3,3,스파이더맨: 뉴 유니버스,Movie,Spider-Man: Into the Spider-Verse,171725
4,4,스파이더맨: 홈커밍,Movie,Spider-Man: Homecoming,135874


In [58]:
df2.drop(['Unnamed: 0'], axis=1, inplace=True)

In [59]:
movie_df = df2[df2['type']=='Movie']

In [60]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              150 non-null    int64  
 1   title           150 non-null    object 
 2   type            150 non-null    object 
 3   original_title  112 non-null    object 
 4   idx             146 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 7.0+ KB


In [65]:
list(movie_df[movie_df['idx'].isna()].title)

['부서져 흩어지는 모습을 보여줄게', '신들의 봉우리', '감당 불가 크리스마스', '캘리포니아 크리스마스: 도시의 불빛']

In [66]:
not_in_naver = movie_df[movie_df['idx'].isna()].index
movie_df.drop(not_in_naver, axis=0, inplace=True)

/Users/shimsomin/opt/anaconda3/envs/corona/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [70]:
movie_df['idx'] = movie_df['idx'].astype(int)

/var/folders/4w/4hfq6k852wg71r8sw418pl_40000gn/T/ipykernel_55960/2483503722.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df['idx'] = movie_df['idx'].astype(int)


In [71]:
movie_df.reset_index(drop=True, inplace=True)

In [72]:
movie_df.to_csv(current_dir+"/new_data/movie_in_Naver.csv", index=False)

In [114]:
movie_df[movie_df['title']=='더 킹'].idx = 144314

/Users/shimsomin/opt/anaconda3/envs/corona/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [120]:
movie_df.loc[movie_df['title']=='더 킹' , 'idx'] = 144314

In [121]:
movie_df[movie_df['title']=='더 킹'].idx 

137    144314
Name: idx, dtype: int64

In [123]:
movie_df.to_csv(current_dir+'/new_data/movie_in_Naver.csv', index=False)

In [9]:
idx = [x for x in df['idx'] if np.isnan(x) == False]

In [6]:
titles = list(df['title'].values)

In [54]:
idx = []
driver = webdriver.Chrome(current_dir+"/driver/chromedriver")

for title in titles:
    url = "https://movie.naver.com/movie/search/result.naver?section=movie&query={}".format(title)
    driver.get(url)
    driver.implicitly_wait(3)
    try:
        a = driver.find_element_by_css_selector(".search_list_1 li dl dt a")

        # url 에서 idx 뽑기
        url = a.get_attribute('href')
        idx.append(url.split('=')[-1])
        
    except:
        idx.append(np.nan)
driver.close()

In [11]:
idx= np.array(idx)

In [13]:
idx = idx.astype(np.int)

- 베놈
- 스파이더맨
- 용재호투
- 그녀
- 인생은 아름다워 -> 22126
- 점퍼 -> 64994
- 잭 리처 -> 90834
- 범죄도시 -> 161242
- 어바웃 타임 -> 92075
- 퍼시픽 립 -> 86867
- 수퍼 소닉 -> 172816
- 고스트 버스터즈 (중복 타이틀 존재)
- 나를 찾아줘 -> 116234
- 맘마 미아! -> 66381
- 추격자 -> 68695
- 올드보이 -> 36944
- 해바라기 -> 44529
- 그레이의 50가지 그림자 -> 115303
- 트와일라잇 -> 47229
- 세 얼간이 -> 73372
- 이스케이프 룸 -> 180425
- 수어사이드 스퀴드 -> 126032
- 메멘토 -> 31940
- 리틀 포레스트 -> 154449
- 쥬라기 월드 -> 67786
- 엑시트 -> 174903
- 더 킹 (중복 타이틀 존재)
- 맨 인 블랙
- 쥬라기 공원

In [ ]:
driver = webdriver.Chrome(current_dir+"/driver/chromedriver")

for i in idx[124:]:
    url = "https://movie.naver.com/movie/bi/mi/basic.naver?code={}".format(i)
    driver.get(url)
    time.sleep(2)

In [57]:
df['idx'] = idx

In [64]:
na_idx = df[df['original_title'] == ' '].index
df.loc[na_idx, 'original_title'] = np.nan

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              199 non-null    int64 
 1   title           199 non-null    object
 2   type            199 non-null    object
 3   original_title  134 non-null    object
 4   idx             179 non-null    object
dtypes: int64(1), object(4)
memory usage: 7.9+ KB


In [67]:
df.to_csv(current_dir+'/data/original_title.csv')

### 영화 idx -> 영어 제목 크롤링

In [77]:
def text_normalize(s):
    """
    Arguments
    ---------
    s : str
        Text to normalize
    Returns
    -------
    normalized text. Remove \\n, \\r, \\t, double space
    """

    s = s.replace('&nbsp;', ' ')
    s = s.replace('\xa0', ' ')
    s = normalize_pattern.sub(' ', s)
    s = doublespcae_pattern.sub(' ', s)
    return s.strip()

In [78]:

basic_url_form = 'http://movie.naver.com/movie/bi/mi/basic.nhn?code={}' # idx

def scrap_basic(idx):
    url = basic_url_form.format(idx)
    soup = get_soup(url)
    infomation = {
        'movie_idx': idx,
        'title': title(soup),
        'e_title': e_title(soup)
    }
    return infomation

def title(soup):
    a = soup.select('div[class=mv_info] h3[class=h_movie] a')
    if not a:
        return ''
    return text_normalize(a[0].text)

def e_title(soup):
    strong = soup.select('div[class=mv_info] strong[class=h_movie2]')
    if not strong:
        return ''
    return text_normalize(strong[0].text)

## 평점 크롤러

In [79]:
# idx, type, page
# 공감순
comments_url_form = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={}&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}'

def get_soup(url, headers=None, allow_redirects=True):
    """
    Argument
    --------
    url : str
        Web page url
    headers : dict or None
        Headers
    allow_redirects : Boolean
    Returns
    -------
    bs4.Beautifulsoup format HTML page
    """

    try:
        r = requests.get(url, headers=headers, allow_redirects=allow_redirects).text
        return BeautifulSoup(r, 'lxml')
    except Exception as e:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        traceback_details = {
            'filename': exc_traceback.tb_frame.f_code.co_filename,
            'lineno'  : exc_traceback.tb_lineno,
            'name'    : exc_traceback.tb_frame.f_code.co_name,
            'type'    : exc_type.__name__,
            'message' : str(e)
        }
        pprint(traceback_details)
        return ''

def scrap_comments(idx, limit=-1, sleep=0.05, last_time=None, i_movie=-1, n_total_movies=-1):
    max_page = num_of_comment_pages(idx)
    if limit > 0:
        max_page = min(limit, max_page)
    if max_page <= 0:
        return []

    if n_total_movies < 0 or i_movie < 0:
        desc = f'Scrap comments {idx}'
    else:
        desc = f'Scrap comments {idx} ({i_movie}/{n_total_movies})'

    comments = []
    for p in trange(1, max_page + 1, desc=desc):
        url = comments_url_form.format(idx, p)
        comments_p, stop = parse_a_page(get_soup(url), last_time)
        comments += comments_p
        if stop:
            print(f'\r  movie {idx}. stop scrap comments. found existing comments {p} / {max_page}')
            break
    return comments[::-1]

def parse_a_page(soup, last_time=None):
    comments = []
    stop = False
    for row in soup.select('div[class=score_result] li'):
        try:
            score = int(row.select('div[class=star_score] em')[0].text.strip())
            text = row.select('div[class=score_reple] p')[0].text.strip()
            # detach '관람객' icon
            if text[:4] == '관람객\n':
                text = text[4:].strip()
            # detach '스포일러' icon
            if text[:25] == '스포일러가 포함된 감상평입니다. 감상평 보기\n':
                text = text[25:].strip()

            comments.append(
                {'score': score,
                 'text': text
                })
        except Exception as e:
            continue
    return comments, stop

def num_of_comment_pages(idx):
    url = comments_url_form.format(idx, 1)
    soup = get_soup(url)

    try:
        num_comments = int(soup.select('div[class="score_total"] em')[-1].text.replace(',',''))
        return math.ceil(num_comments / 5)
    except Exception as e:
        return -1

In [80]:
current_dir = os.getcwd()

df = pd.read_csv(current_dir+"/new_data/movie_in_Naver.csv")
df.head()

,id,title,type,original_title,idx
0,0,모가디슈,Movie,NaN,192150
1,1,베놈,Movie,Venom,119428
2,2,스파이더맨,Movie,Spider-Man,31307
3,3,스파이더맨: 뉴 유니버스,Movie,Spider-Man: Into the Spider-Verse,171725
4,4,스파이더맨: 홈커밍,Movie,Spider-Man: Homecoming,135874


In [85]:
scrap_reviews = scrap_comments(31307, limit=10)

Scrap comments 31307: 100%|██████████| 10/10 [00:00<00:00, 12.15it/s]


In [180]:
merge_df2 = pd.DataFrame(columns=['idx', 'score', 'text'])

for id in concat_df.tail(13).idx:
    scrap_reviews = pd.DataFrame(scrap_comments(id, limit=100))
    scrap_reviews['idx'] = id
    merge_df = pd.concat([merge_df, scrap_reviews], axis=0)

Scrap comments 144314: 100%|██████████| 100/100 [00:09<00:00, 10.79it/s]


In [189]:
merge_df.reset_index(drop=True, inplace=True)

In [190]:
merge_df

,idx,score,text
0,192150,1,정만식이 페미 옹호한다고 남자들한테 욕 쓴게 자꾸 생각나네... 만식아... 페미 ...
1,192150,9,역사적인 내용과 스토리 괜찮았음 볼만함
2,192150,8,신파없는 영화라서 너무 좋았어요 연기력과 연출력 만으로 이렇게 재밋게 만들수 있는데...
3,192150,10,정말 잘짜여진 영화같습니다지루할틈 없이 잘봤습니다!!
4,192150,1,페미들이 평점 조작한다는 화력지원으로 점수 올린다고해서 정상점수 만들기 운동에서 와슴
...,...,...,...
127595,144314,10,요즘 시국에 진짜 잘맞는... 최고의 영화..!
127596,144314,9,대한민국의 왕은 국민. 투표 잊지 맙시다.
127597,144314,10,엔딩크래딧이 블랙리스트인 수준의 영화.
127598,144314,10,스스로 내가 역사고 이 나라라고 생각하는 모든 윗대가리들이 봐야할 영화


In [192]:
merge_df.to_csv(current_dir+'/new_data/movie_reviews.csv', index=False)

## restart

In [202]:
merge_df

,idx,score,text
0,192150,1,정만식이 페미 옹호한다고 남자들한테 욕 쓴게 자꾸 생각나네... 만식아... 페미 ...
1,192150,9,역사적인 내용과 스토리 괜찮았음 볼만함
2,192150,8,신파없는 영화라서 너무 좋았어요 연기력과 연출력 만으로 이렇게 재밋게 만들수 있는데...
3,192150,10,정말 잘짜여진 영화같습니다지루할틈 없이 잘봤습니다!!
4,192150,1,페미들이 평점 조작한다는 화력지원으로 점수 올린다고해서 정상점수 만들기 운동에서 와슴
...,...,...,...
127595,144314,10,요즘 시국에 진짜 잘맞는... 최고의 영화..!
127596,144314,9,대한민국의 왕은 국민. 투표 잊지 맙시다.
127597,144314,10,엔딩크래딧이 블랙리스트인 수준의 영화.
127598,144314,10,스스로 내가 역사고 이 나라라고 생각하는 모든 윗대가리들이 봐야할 영화


In [218]:
df2.head()

,id,title,type,original_title,idx
0,0,모가디슈,Movie,NaN,192150
1,1,베놈,Movie,Venom,119428
2,2,스파이더맨,Movie,Spider-Man,31307
3,3,스파이더맨: 뉴 유니버스,Movie,Spider-Man: Into the Spider-Verse,171725
4,4,스파이더맨: 홈커밍,Movie,Spider-Man: Homecoming,135874


In [204]:
len(id_title['idx'].unique())

145

In [216]:
id_title[id_title['title']=='더 킹']

,id,title,idx
137,230,더 킹,187526


In [224]:
df2_ = df2[~df2['id'].isin(not_existence.id)]

In [225]:
concat_df2 = pd.concat([df2_, not_existence])

In [230]:
concat_df2 = concat_df2.drop(76)

In [231]:
concat_df2['idx'] = concat_df2['idx'].astype(int)

/var/folders/4w/4hfq6k852wg71r8sw418pl_40000gn/T/ipykernel_55960/31593135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concat_df2['idx'] = concat_df2['idx'].astype(int)


In [232]:
concat_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145 entries, 0 to 133
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              145 non-null    int64 
 1   title           145 non-null    object
 2   type            145 non-null    object
 3   original_title  107 non-null    object
 4   idx             145 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 6.8+ KB


In [248]:
dictionary = {
    '이름':['김','이', '박']
    }

In [249]:
df = pd.DataFrame(dictionary)

In [254]:
type(df['이름'])

pandas.core.series.Series

In [255]:
type(dictionary)

dict

In [233]:
id_title = concat_df2[['id', 'title', 'idx']]

In [205]:
len(merge_df['idx'].unique())

142

In [239]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127600 entries, 0 to 127599
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   idx     127600 non-null  object
 1   score   127600 non-null  object
 2   text    127600 non-null  object
dtypes: object(3)
memory usage: 2.9+ MB


In [234]:
final_df = pd.merge(id_title, merge_df, on='idx', how='right')

In [235]:
len(final_df['idx'].unique())

142

In [236]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127600 entries, 0 to 127599
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      127600 non-null  int64 
 1   title   127600 non-null  object
 2   idx     127600 non-null  object
 3   score   127600 non-null  object
 4   text    127600 non-null  object
dtypes: int64(1), object(4)
memory usage: 5.8+ MB


In [237]:
len(final_df.id.unique())

142

In [240]:
final_df.to_csv(current_dir+'/new_data/id_reviews_final.csv',index=False)

In [242]:
list(df2[~df2['id'].isin(final_df.id.unique())].title)

['세븐틴 어게인', '하프 브라더스', '마스터 클렌즈', '로미오와 줄리엣']

In [198]:
not_existence

,id,title,type,original_title,idx
21,30,노트북,Movie,The Notebook,38899.0
23,33,러브 액츄얼리,Movie,Love Actually,36843.0
41,63,인터스텔라,Movie,Interstellar,45290.0
51,81,세븐틴 어게인,Movie,17 Again,52517.0
55,86,하프 브라더스,Movie,Half Brothers,200587.0
57,88,마스터 클렌즈,Movie,The Cleanse,127392.0
68,101,노인을 위한 나라는 없다,Movie,No Country for Old Men,66751.0
73,111,거인,Movie,NaN,117061.0
76,114,로미오와 줄리엣,Movie,Romeo + Juliet,NaN
95,139,비포 선라이즈,Movie,Before Sunrise,17773.0


In [103]:
df.head(7)

,id,title,type,original_title,idx
0,0,모가디슈,Movie,NaN,192150
1,1,베놈,Movie,Venom,119428
2,2,스파이더맨,Movie,Spider-Man,31307
3,3,스파이더맨: 뉴 유니버스,Movie,Spider-Man: Into the Spider-Verse,171725
4,4,스파이더맨: 홈커밍,Movie,Spider-Man: Homecoming,135874
5,5,파워 오브 도그,Movie,The Power of the Dog,193857
6,6,돈 룩 업,Movie,Don't Look Up,192614


In [172]:
merge_df

,idx,score,text
0,192150,1,정만식이 페미 옹호한다고 남자들한테 욕 쓴게 자꾸 생각나네... 만식아... 페미 ...
1,192150,9,역사적인 내용과 스토리 괜찮았음 볼만함
2,192150,8,신파없는 영화라서 너무 좋았어요 연기력과 연출력 만으로 이렇게 재밋게 만들수 있는데...
3,192150,10,정말 잘짜여진 영화같습니다지루할틈 없이 잘봤습니다!!
4,192150,1,페미들이 평점 조작한다는 화력지원으로 점수 올린다고해서 정상점수 만들기 운동에서 와슴
...,...,...,...
117649,13500,10,재개봉이라 날로먹는다까는분들 이영화는 애초 3D제작 영화가 아니라 3D변환할때 겹쳐...
117650,13500,10,20년전 영화가 맨오브스틸보다 훨 나음.
117651,13500,10,재개봉 한다고 1점 주는 넘들은 뭐냐? 볼 사람은 보고 안 볼사람은 안보면 되지. ...
117652,13500,10,최고다!! 20년전 영화인데도.. CG라던가.. 공룡들이 진짜 마치 실사같다~와~ ...


In [113]:
df[df['idx']==187526]

,id,title,type,original_title,idx
79,117,더 킹: 헨리 5세,Movie,The King,187526
137,230,더 킹,Movie,NaN,187526


In [124]:
df[df.duplicated(['idx'])]

,id,title,type,original_title,idx
137,230,더 킹,Movie,NaN,187526


In [129]:
not_existence = df[~df['idx'].isin(merge_idx)]

In [147]:
idx = df[df['title'].isin(list(not_existence.title))].index

In [149]:
df.drop(idx, axis=0, inplace=True)

In [166]:
concat_df = pd.concat([df, not_existence])

In [167]:
concat_df[concat_df['title']=='로미오와 줄리엣']

,id,title,type,original_title,idx
76,114,로미오와 줄리엣,Movie,Romeo + Juliet,NaN


In [168]:
concat_df = concat_df.drop(76)

In [169]:
concat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145 entries, 0 to 133
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              145 non-null    int64  
 1   title           145 non-null    object 
 2   type            145 non-null    object 
 3   original_title  107 non-null    object 
 4   idx             145 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 6.8+ KB


In [258]:
concat_df.to_csv(current_dir+"/new_data/movie_in_Naver.csv", index=False)

In [257]:
concat_df

,id,title,type,original_title,idx
0,0,모가디슈,Movie,NaN,192150
1,1,베놈,Movie,Venom,119428
2,2,스파이더맨,Movie,Spider-Man,31307
3,3,스파이더맨: 뉴 유니버스,Movie,Spider-Man: Into the Spider-Verse,171725
4,4,스파이더맨: 홈커밍,Movie,Spider-Man: Homecoming,135874
...,...,...,...,...,...
95,139,비포 선라이즈,Movie,Before Sunrise,17773
101,147,악마를 보았다,Movie,NaN,72408
115,169,가디언즈,Movie,Rise of the Guardians,87308
132,225,엑시트,Movie,NaN,174903


In [175]:
concat_df.idx = concat_df.idx.astype(int)

In [179]:
concat_df.tail(13).idx

21      38899
23      36843
41      45290
51      52517
55     200587
57     127392
68      66751
73     117061
95      17773
101     72408
115     87308
132    174903
133    144314
Name: idx, dtype: int64

In [173]:
not_existence

,id,title,type,original_title,idx
21,30,노트북,Movie,The Notebook,38899.0
23,33,러브 액츄얼리,Movie,Love Actually,36843.0
41,63,인터스텔라,Movie,Interstellar,45290.0
51,81,세븐틴 어게인,Movie,17 Again,52517.0
55,86,하프 브라더스,Movie,Half Brothers,200587.0
57,88,마스터 클렌즈,Movie,The Cleanse,127392.0
68,101,노인을 위한 나라는 없다,Movie,No Country for Old Men,66751.0
73,111,거인,Movie,NaN,117061.0
76,114,로미오와 줄리엣,Movie,Romeo + Juliet,NaN
95,139,비포 선라이즈,Movie,Before Sunrise,17773.0


In [136]:
not_existence.loc[133] = [230, '더 킹',	'Movie', np.nan, 144314]

/Users/shimsomin/opt/anaconda3/envs/corona/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [140]:
not_existence.shape

(14, 5)

In [138]:
modified_idx = [38899, 36843, 45290, 52517, 200587, 127392, 66751, 117061, 17773, 72408, 87308, 174903, 144314]

In [139]:
len(modified_idx)

14

In [142]:
not_existence['idx'] = modified_idx

/var/folders/4w/4hfq6k852wg71r8sw418pl_40000gn/T/ipykernel_55960/762430231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_existence['idx'] = modified_idx


In [143]:
not_existence

,id,title,type,original_title,idx
21,30,노트북,Movie,The Notebook,38899.0
23,33,러브 액츄얼리,Movie,Love Actually,36843.0
41,63,인터스텔라,Movie,Interstellar,45290.0
51,81,세븐틴 어게인,Movie,17 Again,52517.0
55,86,하프 브라더스,Movie,Half Brothers,200587.0
57,88,마스터 클렌즈,Movie,The Cleanse,127392.0
68,101,노인을 위한 나라는 없다,Movie,No Country for Old Men,66751.0
73,111,거인,Movie,NaN,117061.0
76,114,로미오와 줄리엣,Movie,Romeo + Juliet,NaN
95,139,비포 선라이즈,Movie,Before Sunrise,17773.0


In [98]:
merge_df.reset_index(drop=True, inplace=True)

In [99]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117654 entries, 0 to 117653
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   idx     117654 non-null  object
 1   score   117654 non-null  object
 2   text    117654 non-null  object
dtypes: object(3)
memory usage: 2.7+ MB


In [100]:
merge_df

,idx,score,text
0,192150,1,정만식이 페미 옹호한다고 남자들한테 욕 쓴게 자꾸 생각나네... 만식아... 페미 ...
1,192150,9,역사적인 내용과 스토리 괜찮았음 볼만함
2,192150,8,신파없는 영화라서 너무 좋았어요 연기력과 연출력 만으로 이렇게 재밋게 만들수 있는데...
3,192150,10,정말 잘짜여진 영화같습니다지루할틈 없이 잘봤습니다!!
4,192150,1,페미들이 평점 조작한다는 화력지원으로 점수 올린다고해서 정상점수 만들기 운동에서 와슴
...,...,...,...
117649,13500,10,재개봉이라 날로먹는다까는분들 이영화는 애초 3D제작 영화가 아니라 3D변환할때 겹쳐...
117650,13500,10,20년전 영화가 맨오브스틸보다 훨 나음.
117651,13500,10,재개봉 한다고 1점 주는 넘들은 뭐냐? 볼 사람은 보고 안 볼사람은 안보면 되지. ...
117652,13500,10,최고다!! 20년전 영화인데도.. CG라던가.. 공룡들이 진짜 마치 실사같다~와~ ...


In [126]:
merge_idx = list(merge_df['idx'].unique())

In [114]:
merge_df.shape

(106943, 3)

In [116]:
result_df = pd.merge(df, merge_df, left_on='idx', right_on='movie_idx', how='outer')

In [124]:
len(result_df[result_df['movie_idx'].isna()].title.unique())

68

In [121]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112841 entries, 0 to 112840
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              112841 non-null  int64 
 1   title           112841 non-null  object
 2   type            112841 non-null  object
 3   original_title  83489 non-null   object
 4   idx             112821 non-null  object
 5   movie_idx       112773 non-null  object
 6   score           112773 non-null  object
 7   text            112773 non-null  object
dtypes: int64(1), object(7)
memory usage: 7.7+ MB


In [120]:
df[df['idx'].isna()].shape

(20, 5)

In [125]:
result_df.to_csv(current_dir+'/data/naver_movie_reviews.csv', index=False)